In [8]:
import pandas as pd

In [9]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None) 
df_final = pd.read_csv("../data/F_feature_selection/feature_selection-2022.csv", sep=";")
display(df_final.head())
print(len(df_final.columns))

,location,type_of_collision,latitude,longitude,reserved_lane_present,horizontal_alignment,infrastructure,accident_situation,speed_limit,position,sex,pedestrian_location,pedestrian_action,fixed_obstacle_struck,mobile_obstacle_struck,initial_point_of_impact,main_maneuver_before_accident,motor_type,fixed_obstacle_struck_other,mobile_obstacle_struck_other,initial_point_of_impact_other,main_maneuver_before_accident_other,motor_type_other,vehicle_category_involved_bicycle,vehicle_category_involved_bus_coach,vehicle_category_involved_hgv_truck,vehicle_category_involved_light_motor_vehicle,vehicle_category_involved_other,vehicle_category_involved_powered_2_3_wheeler,hour_sin,hour_cos,time_of_day,day_of_week,day_of_week_sin,day_of_week_cos,month_sin,month_cos,day_of_year_sin,day_of_year_cos,age,age_group,used_belt,used_helmet,used_child_restraint,used_airbag,vehicle_category_simplified,impact_score,vehicle_category_simplified_other,impact_score_other,impact_delta,road_complexity_index,surface_quality_indicator,role,lighting_ordinal,weather_ordinal,injury_target,cluster
0,2,3,44.559420,4.725720,0,1,0,1,50,1,1,-1,-1,0,2,1,9,1,0,2,2,1,1,0,0,0,1,0,1,-0.866025,-0.500000,Evening_Rush,2,0.974928,-0.222521,-0.866025,0.5,-0.952114,0.305743,14,child_teen,0,1,0,0,powered_2_3_wheeler,3,light_motor_vehicle,4,-1,5.0,1,driver,0,0,2,1
1,2,3,44.559420,4.725720,0,1,0,1,50,1,1,-1,-1,0,2,2,1,1,0,2,1,9,1,0,0,0,1,0,1,-0.866025,-0.500000,Evening_Rush,2,0.974928,-0.222521,-0.866025,0.5,-0.952114,0.305743,74,senior,1,0,0,0,light_motor_vehicle,4,powered_2_3_wheeler,3,1,5.0,1,driver,0,0,0,1
2,2,3,46.925810,6.346200,0,1,0,1,50,1,1,0,0,0,2,8,15,1,0,2,1,1,1,0,0,0,2,0,0,0.866025,-0.500000,Morning_Rush,3,0.433884,-0.900969,-0.866025,0.5,-0.946714,0.322076,34,adult,1,0,0,0,light_motor_vehicle,4,light_motor_vehicle,4,0,5.0,1,driver,0,0,1,1
3,2,3,46.925810,6.346200,0,1,0,1,50,1,1,0,0,0,2,1,1,1,0,2,8,15,1,0,0,0,2,0,0,0.866025,-0.500000,Morning_Rush,3,0.433884,-0.900969,-0.866025,0.5,-0.946714,0.322076,52,middle_aged,1,0,0,0,light_motor_vehicle,4,light_motor_vehicle,4,0,5.0,1,driver,0,0,0,1
4,2,2,48.493162,-2.760439,0,1,5,1,50,1,1,-1,-1,0,2,1,2,1,0,2,4,2,1,0,0,0,2,0,0,-0.965926,-0.258819,Evening_Rush,3,0.433884,-0.900969,-0.866025,0.5,-0.946714,0.322076,20,young_adult,1,0,0,0,light_motor_vehicle,4,light_motor_vehicle,4,0,5.0,1,driver,0,0,0,1


57


In [10]:
df_sample = df_final.sample(10000, random_state=42)
targets = df_final['injury_target']
targets_sample = df_sample['injury_target']
df_final.drop(['injury_target', 'cluster'], axis=1, inplace=True)
df_sample.drop(['injury_target', 'cluster'], axis=1, inplace=True)

### Identifying Numerical, Ordinal, and Categorical Columns

In this step, three groups of features for clustering are defined:

- **`num_cols`**: Numerical variables, including speed limit, vehicle category indicators, road complexity index, and impact delta.
- **`ord_cols`**: Ordinal variables that represent ordered categories (e.g., lighting conditions, weather).
- **`cat_cols`**: All remaining columns that are neither numerical nor ordinal. These are identified programmatically to avoid manual listing.

In [11]:

num_cols = ['latitude', 'longitude', 'speed_limit', 'age', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', "month_sin", "month_cos", "day_of_year_sin", "day_of_year_cos"]
ord_cols = ['lighting_ordinal', 'weather_ordinal', 'road_complexity_index', 'impact_score', 'impact_score_other', 'impact_delta']
cat_cols = []

for col in df_final.columns:
    if col not in num_cols and col not in ord_cols:
        cat_cols.append(col)
print(cat_cols)

['location', 'type_of_collision', 'reserved_lane_present', 'horizontal_alignment', 'infrastructure', 'accident_situation', 'position', 'sex', 'pedestrian_location', 'pedestrian_action', 'fixed_obstacle_struck', 'mobile_obstacle_struck', 'initial_point_of_impact', 'main_maneuver_before_accident', 'motor_type', 'fixed_obstacle_struck_other', 'mobile_obstacle_struck_other', 'initial_point_of_impact_other', 'main_maneuver_before_accident_other', 'motor_type_other', 'vehicle_category_involved_bicycle', 'vehicle_category_involved_bus_coach', 'vehicle_category_involved_hgv_truck', 'vehicle_category_involved_light_motor_vehicle', 'vehicle_category_involved_other', 'vehicle_category_involved_powered_2_3_wheeler', 'time_of_day', 'day_of_week', 'age_group', 'used_belt', 'used_helmet', 'used_child_restraint', 'used_airbag', 'vehicle_category_simplified', 'vehicle_category_simplified_other', 'surface_quality_indicator', 'role']


## Preprocessing Components

To properly prepare each feature type, the following scikit-learn transformers are used:

- **OneHotEncoder (`cat_encoder`)**  
  - Used for categorical variables.  
  - `drop="first"` prevents multicollinearity by dropping the first category.  
  - `handle_unknown="ignore"` ensures that unseen categories during inference will not cause errors.

- **OrdinalEncoder (`ord_encoder`)**  
  - Transforms ordered categories (e.g., weather, lighting) into integer rankings while preserving order relations.

- **StandardScaler (`num_scaler` & `ord_scaler`)**  
  - Scales numerical and ordinal features to have mean 0 and standard deviation 1.  
  - Ensures that all features contribute equally to distance-based algorithms like K-Means.


In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_encoder = OneHotEncoder(drop="first", handle_unknown="ignore")
ord_encoder = OrdinalEncoder()
num_scaler = StandardScaler()
ord_scaler = StandardScaler()

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("scaler", num_scaler)
        ]), num_cols),
        ("ord", Pipeline([
            ("encoder", ord_encoder),
            ("scaler", ord_scaler)
        ]), ord_cols),
        ("cat", Pipeline([
            ("encoder", cat_encoder)
        ]), cat_cols)
    ]
)

df_prepared = preprocess.fit_transform(df_final)
df_prepared_sample = preprocess.fit_transform(df_sample)

In [13]:
df_eval = df_sample.copy()
for col in cat_cols:
    df_eval[col] = df_eval[col].astype(str)
#df['age_group'] = df['age_group'].astype(str)
df_np = df_eval.to_numpy()
cat_indices = [df_final.columns.get_loc(col) for col in cat_cols]

In [14]:
results = {
    "algorithm": [],
    "parameter": [],
    "labels": [],
    "silhouette": []
}

best_result = {
    "algorithm": None,
    "parameter": None,
    "labels": None,
    "silhouette": -1
}

In [15]:
ns_clusters = [3, 5, 7]
min_cluster_sizes = [10, 25, 50]

## K-Prototypes clustering

K-Prototypes is suitable for datasets containing both numerical and categorical variables.  
Before running it, we need to separate numerical and categorical features and ensure the data is in the correct format (especially converting categories to strings, as required by the algorithm).

In the next steps, we will prepare the dataset accordingly and run K-Prototypes on the mixed-feature data.

In [36]:
from sklearn.metrics import silhouette_score
# K-PROTOTYPES
from kmodes.kprototypes import KPrototypes


for k in ns_clusters:
    kproto = KPrototypes(
        n_clusters=k,
        init='Huang',
        n_init=5,
        verbose=0,
        random_state=42
    )

    labels = kproto.fit_predict(df_np, categorical=cat_indices)
    sil = silhouette_score(df_prepared_sample, labels)  # df_prepared = numeric representation

    # Save in dictionary
    results["algorithm"].append("K-Prototypes")
    results["parameter"].append(k)
    results["labels"].append(labels)
    results["silhouette"].append(sil)

    # Track best
    if sil > best_result["silhouette"]:
        best_result["algorithm"] = "K-Prototypes"
        best_result["parameter"] = k
        best_result["labels"] = labels
        best_result["silhouette"] = sil

## Hierarchical Clustering with Gower Distance

For datasets containing both numerical and categorical features, **Gower distance** provides a way to compute similarities between mixed-type records.  
Then, **Agglomerative Hierarchical Clustering** and **HDBSCAN** using this distance matrix.

Since hierarchical clustering can be computationally expensive on large datasets, a sample of 1,000 records is taken.

In [37]:
import gower
from sklearn.cluster import AgglomerativeClustering


gower_dist = gower.gower_matrix(df_sample)

In [38]:
# AGGLOMERATIVE CLUSTERING

for k in ns_clusters:
    agg = AgglomerativeClustering(
        n_clusters=k,
        metric="precomputed",
        linkage="average"
    )

    labels = agg.fit_predict(gower_dist)

    sil = silhouette_score(df_prepared_sample, labels)

    results["algorithm"].append("Agglomerative")
    results["parameter"].append(k)
    results["labels"].append(labels)
    results["silhouette"].append(sil)

    if sil > best_result["silhouette"]:
        best_result["algorithm"] = "Agglomerative"
        best_result["parameter"] = k
        best_result["labels"] = labels
        best_result["silhouette"] = sil

In [39]:
import hdbscan
import numpy as np

for mcs in min_cluster_sizes:
    hdb = hdbscan.HDBSCAN(
        min_cluster_size=mcs,
        metric="precomputed"
    )

    labels = hdb.fit_predict(gower_dist.astype(np.float64))

    # HDBSCAN can assign many -1 noise points
    # only compute silhouette if at least 2 clusters exist
    #if len(set(labels)) > 1 and -1 not in set(labels):
    sil = silhouette_score(df_prepared_sample, labels)
    #else:
    #    sil = -1  # Not valid or too much noise

    results["algorithm"].append("HDBSCAN")
    results["parameter"].append(mcs)
    results["labels"].append(labels)
    results["silhouette"].append(sil)

    if sil > best_result["silhouette"]:
        best_result["algorithm"] = "HDBSCAN"
        best_result["parameter"] = mcs
        best_result["labels"] = labels
        best_result["silhouette"] = sil

## Check what algorithm works best

In [40]:
results_df = pd.DataFrame({
    "algorithm": results["algorithm"],
    "parameter": results["parameter"],
    "silhouette": results["silhouette"],
    "parameter": results["parameter"]
})

results_df


,algorithm,parameter,silhouette
0,K-Prototypes,3,0.032305
1,K-Prototypes,5,0.040767
2,K-Prototypes,7,0.026212
3,Agglomerative,3,0.107160
4,Agglomerative,5,0.072454
5,Agglomerative,7,0.062951
6,HDBSCAN,10,0.088780
7,HDBSCAN,25,0.070354
8,HDBSCAN,50,0.048149


In [16]:
best_results = results_df.loc[
    results_df.groupby("algorithm")["silhouette"].idxmax()
]

best_results

,algorithm,parameter,silhouette
6,Agglomerative,3,0.239700
9,HDBSCAN,10,0.088715
0,K-Means,3,0.066564
4,K-Prototypes,5,0.046814


Agglomerative hierarchical clustering as well as HDBSCAN seem to be the best algorithm in this case. 

In [41]:
for algo, param, labels in zip(results["algorithm"], results["parameter"], results["labels"]):
    #if algo != "kmeans" and algo != "kproto":
    # Construct column name
    colname = f"cluster_{algo}_{param}"
    # Add labels as new column
    df_sample[colname] = labels

In [42]:
cluster_sizes = {}

# Loop over all cluster columns
cluster_cols = [col for col in df_sample.columns if col.startswith("cluster_")]

for col in cluster_cols:
    sizes = df_sample[col].value_counts().sort_index()
    cluster_sizes[col] = sizes

# Convert to DataFrame for easy viewing
cluster_sizes_df = pd.DataFrame(cluster_sizes).fillna(0).astype(int)
cluster_sizes_df

,cluster_K-Prototypes_3,cluster_K-Prototypes_5,cluster_K-Prototypes_7,cluster_Agglomerative_3,cluster_Agglomerative_5,cluster_Agglomerative_7,cluster_HDBSCAN_10,cluster_HDBSCAN_25,cluster_HDBSCAN_50
-1,0,0,0,0,0,0,759,1762,3144
0,4143,295,1739,9996,9252,9246,8542,7596,6295
1,2625,3084,1178,1,741,741,699,642,561
2,3232,233,295,3,3,4,0,0,0
3,0,2509,656,0,1,1,0,0,0
4,0,3879,3631,0,3,3,0,0,0
5,0,0,233,0,0,3,0,0,0
6,0,0,2268,0,0,2,0,0,0


In [43]:
best_cluster_cols = {}

for _, row in best_results.iterrows():
    algo = row["algorithm"]
    param = str(row["parameter"])  # ensure string match

    # Find the correct cluster column
    for col in cluster_cols:
        if algo in col and param in col:
            best_cluster_cols[algo] = col
            break

In [44]:
compact_stats = []

for algo, col in best_cluster_cols.items():
    counts = df_sample[col].value_counts()
    total = counts.sum()
    perc = (counts / total * 100).sort_values(ascending=False)

    top1 = perc.iloc[0]
    top2 = perc.iloc[1] if len(perc) > 1 else 0
    top3 = perc.iloc[2] if len(perc) > 2 else 0

    compact_stats.append({
        "algorithm": algo,
        "largest_cluster_%": round(top1, 1),
        #"second_largest_%": round(top2, 1),
        #"third_largest_%": round(top3, 1)
    })

compact_df = pd.DataFrame(compact_stats)

In [21]:
final_df = best_results.merge(compact_df, on="algorithm")
final_df

,algorithm,parameter,silhouette,largest_cluster_%,second_largest_%,third_largest_%
0,Agglomerative,3,0.239700,99.9,0.0,0.0
1,HDBSCAN,10,0.088715,84.3,8.8,6.9
2,K-Means,3,0.066564,39.3,36.2,24.5
3,K-Prototypes,5,0.046814,38.9,31.1,24.8


Based off largest cluster and silhouette scores, K-Prototypes with k=5 is selected as final clustering algorithm. Since K-Prototypes can handle more data than Agglomerative and HDBSCAN, run the clustering again on a larger corpus to capture more of the data.

In [16]:
from kmodes.kprototypes import KPrototypes
df_eval_whole = df_final.copy()
for col in cat_cols:
    df_eval_whole[col] = df_eval_whole[col].astype(str)
df_np_whole = df_eval_whole.to_numpy()

kproto = KPrototypes(
        n_clusters=5,
        init='Huang',
        n_init=5,
        verbose=0,
        random_state=42
)

labels = kproto.fit_predict(df_np_whole, categorical=cat_indices)
df_clustered = df_final.copy()
df_clustered["cluster_K-Prototypes_5"] = labels

## Interpreting clusters

In [23]:
pd.crosstab(df_sample["cluster_K-Prototypes_5"], targets_sample)

injury_target,0,1,2
cluster_K-Prototypes_5,,,
0,1725,1698,464
1,1127,886,465
2,1234,1148,725
3,114,113,68
4,98,77,58


## Create Accident profiles

In [46]:
for col in cat_cols:
    print(f"\nFeature: {col}")
    
    # Compute contingency table
    contingency = pd.crosstab(df_sample["cluster_K-Prototypes_5"], df_sample[col])
    
    # Normalize rows to get proportions per cluster
    contingency_norm = contingency.div(contingency.sum(axis=1), axis=0)
    
    print(contingency_norm)


Feature: location
location                       1         2
cluster_K-Prototypes_5                    
0                       0.488136  0.511864
1                       0.927367  0.072633
2                       0.343348  0.656652
3                       0.186130  0.813870
4                       0.072441  0.927559

Feature: type_of_collision
type_of_collision             -1         1         2         3         4  \
cluster_K-Prototypes_5                                                     
0                       0.000000  0.128814  0.125424  0.322034  0.061017   
1                       0.000324  0.142672  0.213359  0.190337  0.127432   
2                       0.000000  0.133047  0.103004  0.339056  0.017167   
3                       0.000797  0.111598  0.129932  0.349940  0.044241   
4                       0.000516  0.102346  0.118072  0.404228  0.034803   

type_of_collision              5         6         7  
cluster_K-Prototypes_5                                
0        

In [11]:
# Step 0: Configuration (tune these if needed)
cluster_col = "cluster_K-Prototypes_5"   # the cluster column you want to explain
lift_threshold = 1.2                     # include if cluster proportion >= lift_threshold * global proportion
support_threshold = 0.05                 # include only if category appears in at least 5% of the cluster
variance_threshold = 0.005                # drop features where proportions are too flat across clusters

# Step 1: Basic contingency and normalized contingency
# This computes counts and row-normalized proportions per cluster for each categorical feature.
contingencies = {}        # raw counts per feature
contingencies_norm = {}   # P(category | cluster) per feature

for col in cat_cols:
    contingency = pd.crosstab(df_clustered[cluster_col], df_clustered[col])
    contingency_norm = contingency.div(contingency.sum(axis=1), axis=0).fillna(0)
    contingencies[col] = contingency
    contingencies_norm[col] = contingency_norm

# Step 2: Solution 2 filter — drop features where the dominant category is identical across (most) clusters

# We consider a feature "trivial" if the most frequent category (argmax) is the same
# for all clusters (or for a large majority of clusters). Here we do the strict version:
# drop feature if the dominant category is identical for *all* clusters.
useful_features_step2 = []
for col, cont in contingencies.items():
    # dominant category per cluster (could be ties; take first)
    dominant_per_cluster = cont.idxmax(axis=1).tolist()
    # if not every element is equal, keep the feature
    if len(set(dominant_per_cluster)) > 1:
        useful_features_step2.append(col)

# Step 3: Solution 3 filter — variance threshold

# Drop features whose category-proportions vary very little across clusters.
useful_features_step3 = []
for col in useful_features_step2:
    prop = contingencies_norm[col]  # DataFrame: index=cluster, columns=categories
    # compute std dev across clusters for each category, then take max std across categories
    # If all categories have near-zero std, feature is flat.
    stds = prop.std(axis=0)
    max_std = stds.max() if not stds.empty else 0.0
    if max_std > variance_threshold:
        useful_features_step3.append(col)

# Step 4: Solution 1 + support filter — compute lift and keep overrepresented + supported categories
accident_profiles = {}

for col in useful_features_step3:
    prop = contingencies_norm[col]                       # P(category | cluster)
    global_freq = df_sample[col].value_counts(normalize=True)  # P(category)
    
    # iterate clusters and categories
    for cluster_id in prop.index:
        # prepare container
        accident_profiles.setdefault(cluster_id, {})
        
        for category in prop.columns:
            p_cluster = prop.loc[cluster_id, category]   # P(cat | cluster)
            p_global = global_freq.get(category, 0.0)   # P(cat)
            
            # skip if global frequency is zero (shouldn't happen normally)
            if p_global == 0:
                continue
            
            lift = p_cluster / p_global
            support = p_cluster  # proportion inside cluster
            
            # apply both lift and support thresholds
            if (lift >= lift_threshold) and (support >= support_threshold):
                accident_profiles[cluster_id].setdefault(col, [])
                # store category, lift, support (rounded for readability)
                accident_profiles[cluster_id][col].append({
                    "category": category,
                    "lift": round(float(lift), 2),
                    "support_pct": round(float(support * 100), 1)
                })

# Step 5: Optional — clean empty cluster entries (clusters with no overrepresented categories)
accident_profiles = {c: p for c, p in accident_profiles.items() if p}

# Step 6: Pretty-print the results
for cluster_id, profile in sorted(accident_profiles.items()):
    print(f"\nCluster {cluster_id} — selected overrepresented categories:")
    for feature, cats in profile.items():
        cats_str = ", ".join([f"{x['category']} (lift={x['lift']}, {x['support_pct']}%)" for x in cats])
        print(f"  {feature}: {cats_str}")



Cluster 0 — selected overrepresented categories:
  location: 2 (lift=1.22, 75.1%)
  pedestrian_action: 3 (lift=1.73, 9.9%)
  time_of_day: Midday (lift=1.31, 40.2%)
  age_group: middle_aged (lift=1.97, 61.4%), senior (lift=3.57, 38.6%)

Cluster 1 — selected overrepresented categories:
  location: 2 (lift=1.35, 83.3%)
  pedestrian_location: -1 (lift=1.32, 52.7%)
  pedestrian_action:  -1 (lift=1.33, 51.6%)
  time_of_day: Night (lift=1.21, 26.3%)
  age_group: adult (lift=1.51, 43.4%), child_teen (lift=1.53, 14.1%), young_adult (lift=1.63, 29.4%)

Cluster 2 — selected overrepresented categories:
  type_of_collision: 5 (lift=1.26, 6.1%), 7 (lift=1.41, 8.7%)
  time_of_day: Morning_Rush (lift=1.47, 23.5%)
  day_of_week: 5 (lift=1.25, 19.2%)

Cluster 3 — selected overrepresented categories:
  location: 1 (lift=2.55, 97.7%)
  type_of_collision: 2 (lift=1.37, 20.6%), 4 (lift=1.83, 12.1%), 5 (lift=1.94, 9.4%), 7 (lift=1.41, 8.7%)
  pedestrian_location: 0 (lift=1.28, 67.5%)
  pedestrian_action: 0 

In [13]:
numeric_stats = {}

for col in num_cols:
    stats = df_clustered.groupby(cluster_col)[col].agg(
        mean='mean',
        median='median',
        std='std',
        min='min',
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75),
    )
    numeric_stats[col] = stats

In [14]:
numeric_profiles = {}

for col, stats in numeric_stats.items():
    global_mean = df_sample[col].mean()
    global_std = df_sample[col].std()
    
    for cluster_id, row in stats.iterrows():
        numeric_profiles.setdefault(cluster_id, {})
        # compute standardized mean deviation
        z_mean = (row['mean'] - global_mean) / global_std if global_std > 0 else 0
        # also median deviation can be added
        z_median = (row['median'] - df_sample[col].median()) / df_sample[col].std()
        
        numeric_profiles[cluster_id][col] = {
            'mean': round(row['mean'], 2),
            'median': round(row['median'], 2),
            'std': round(row['std'], 2),
            'z_mean': round(z_mean, 2),
            'q25': round(row['q25'], 2),
            'q75': round(row['q75'], 2)
        }


In [15]:
z_threshold = 1.0  # equivalent to 1 std dev away from global mean

highlighted_numerics = {}
for cluster_id, features in numeric_profiles.items():
    highlighted_numerics[cluster_id] = {}
    for feature, stats in features.items():
        if abs(stats['z_mean']) >= z_threshold:
            highlighted_numerics[cluster_id][feature] = stats


In [16]:
cluster_profiles = {}

for cluster_id in set(accident_profiles.keys()).union(highlighted_numerics.keys()):
    cluster_profiles[cluster_id] = {
        'categorical': accident_profiles.get(cluster_id, {}),
        'numeric': highlighted_numerics.get(cluster_id, {})
    }

In [17]:
# Step 7: Pretty-print full cluster profiles

for cluster_id in sorted(cluster_profiles.keys()):
    print(f"\n=== Cluster {cluster_id} Profile ===")
    
    # ---- Categorical features ----
    cat_profile = cluster_profiles[cluster_id].get('categorical', {})
    if cat_profile:
        print("\nCategorical overrepresented features:")
        for feature, cats in cat_profile.items():
            cats_str = ", ".join([f"{x['category']} (lift={x['lift']}, {x['support_pct']}%)" for x in cats])
            print(f"  {feature}: {cats_str}")
    else:
        print("\nNo overrepresented categorical features.")
    
    # ---- Numeric features ----
    num_profile = cluster_profiles[cluster_id].get('numeric', {})
    if num_profile:
        print("\nNumeric features deviating from global average:")
        for feature, stats in num_profile.items():
            print(f"  {feature}: mean={stats['mean']}, median={stats['median']}, std={stats['std']}, "
                  f"q25={stats['q25']}, q75={stats['q75']}, z_mean={stats['z_mean']}")
    else:
        print("\nNo numeric features significantly different from global.")



=== Cluster 0 Profile ===

Categorical overrepresented features:
  location: 2 (lift=1.22, 75.1%)
  pedestrian_action: 3 (lift=1.73, 9.9%)
  time_of_day: Midday (lift=1.31, 40.2%)
  age_group: middle_aged (lift=1.97, 61.4%), senior (lift=3.57, 38.6%)

Numeric features deviating from global average:
  age: mean=62.35, median=60.0, std=12.16, q25=52.0, q75=71.0, z_mean=1.23

=== Cluster 1 Profile ===

Categorical overrepresented features:
  location: 2 (lift=1.35, 83.3%)
  pedestrian_location: -1 (lift=1.32, 52.7%)
  pedestrian_action:  -1 (lift=1.33, 51.6%)
  time_of_day: Night (lift=1.21, 26.3%)
  age_group: adult (lift=1.51, 43.4%), child_teen (lift=1.53, 14.1%), young_adult (lift=1.63, 29.4%)

No numeric features significantly different from global.

=== Cluster 2 Profile ===

Categorical overrepresented features:
  type_of_collision: 5 (lift=1.26, 6.1%), 7 (lift=1.41, 8.7%)
  time_of_day: Morning_Rush (lift=1.47, 23.5%)
  day_of_week: 5 (lift=1.25, 19.2%)

Numeric features deviatin